In [1]:
!pip install scikit-surprise
import numpy as np
import pandas as pd
from surprise import Dataset
from surprise import Reader
import heapq

     |████████████████████████████████| 11.8MB 236kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617634 sha256=f55526f215bfaa59d4ac5a0903acca1b96abdee4fd6ba6adbedf68192fbff185
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
data_main = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz',
                 sep='\t',
                 compression='gzip',
                 error_bad_lines=False,
                 warn_bad_lines=False)

In [3]:
products = data_main[['product_id', 'product_title']].drop_duplicates(['product_id'])
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_main[["customer_id", "product_id", "star_rating"]], reader)
trainingSet = data.build_full_trainset()

In [4]:
from surprise import SVD

algo = SVD(n_epochs=19, lr_all=0.02, reg_all=0.2)
algo.fit(trainingSet)
predictions = algo.test(trainingSet.build_testset())

In [5]:
from surprise.dump import *

In [6]:
dump(file_name='svd.model', predictions=predictions, algo= algo, verbose=1)

The dump has been saved as file svd.model


In [7]:
model = load(file_name='svd.model')

In [8]:
def getProductName(product_id):
      return products.loc[products['product_id'] == product_id, 'product_title'].iloc[0]

In [9]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [10]:
test_customer = 18057691
test_subject_iid = trainingSet.to_inner_uid(test_customer)
test_subject_ratings = trainingSet.ur[test_subject_iid]

In [11]:
k = 10 + len(test_subject_ratings)
watched = {}
for itemID, rating in trainingSet.ur[test_subject_iid]:
  watched[trainingSet.to_raw_iid(itemID)] = 1

In [12]:
list_tem = []
for item in products.product_id:
  list_tem.append(model[1].predict(uid=str(test_customer), iid=item))

In [13]:
k_neighbors = heapq.nlargest(k, list_tem, key=lambda t: t[3])
top_t = get_top_n(k_neighbors, n=k)

In [14]:
position = 0
recommendations = []
for rec in top_t[str(test_customer)]:
  if not rec[0] in watched:
    recommendations.append(getProductName(rec[0]))
    position += 1
    if (position >= 10): break # We only want top 10

In [15]:
for recommendation in recommendations:
  print(recommendation)

USMLE Step 1 Flashcards [Instant Access]
FileCenter Professional 7 for 1 PC [Download]
HitFilm 3 Pro (Win) [Online Code]
Noteflight Crescendo Music Notation Software   Free Trial Available
avast! Internet Security 2014 [Download]
Snagit 12 - Academic [Download]
Avast Free Antivirus 2015 [Download]
IObit Uninstaller 4 [Download]
RPG Maker VX Ace
Intuit QuickBooks Online Simple Start 2015
